# City Life: Data Visualization - 15 Minute City

**DV, AI-Advanced**

In [1]:
import folium
import networkx as nx
import osmnx as ox
from folium.plugins import MeasureControl

In [5]:
def create_15_minute_map(start_location, transport_modes):
    # Initialize a folium map at the start location
    m = folium.Map(location=start_location, zoom_start=13)
    
    # Add a measure control plugin
    m.add_child(MeasureControl())

    # Define travel speeds in km/h
    travel_speed = {'walk': 5, 'bike': 20, 'drive': 40}
    
    # Define colors for each transport mode
    transport_colors = {'walk': 'blue', 'bike': 'orange', 'drive': 'purple'}
    
    # Calculate and add routes for each transport mode
    for mode in transport_modes:
        G = ox.graph_from_point(start_location, network_type=mode, dist=10000, simplify=True)
        center_node = ox.distance.nearest_nodes(G, start_location[1], start_location[0])
        max_distance = travel_speed[mode] * 1000 / 60 * 15  # distance in meters for 15 minutes
        subgraph = nx.ego_graph(G, center_node, radius=max_distance, distance='length')
        
        for node1, node2, data in subgraph.edges(data=True):
            folium.PolyLine([(G.nodes[node1]['y'], G.nodes[node1]['x']),
                             (G.nodes[node2]['y'], G.nodes[node2]['x'])],
                            color=transport_colors[mode], weight=2.5, opacity=1).add_to(m)
    
    return m

# Starting location: PSV Eindhoven stadium
start_location = (51.4416, 5.4697)

# Transport modes to visualize
transport_modes = ['walk', 'bike', 'drive']

# Generate the map
city_map = create_15_minute_map(start_location, transport_modes)

# Save the map to an HTML file
city_map.save('15_minute_city_map.html')